In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [21]:
import autograd_hacks

In [26]:
t = torch.ones([2,3])
print (t.size())
new = torch.ones(t.size())
print (new)

torch.Size([2, 3])
tensor([[1., 1., 1.],
        [1., 1., 1.]])


In [19]:
N, u_dim, z_dim, out_dim = 5, 2, 3, 2
zc = 5 * torch.ones(N, z_dim, requires_grad=True)
uc = 2 * torch.ones(N, u_dim, requires_grad=True)
input = torch.cat((zc, uc), dim=1)
print (input)
layer = nn.Linear(z_dim + u_dim, out_dim)
output = layer(input)
print (output)
zp_predict1 = output[:, 0]
zp_predict2 = output[:, 1]
grad_zp1_zc = torch.autograd.grad(zp_predict1, zc)
print (zc.grad)
zc.grad.zero_()
grad_zp2_zc = torch.autograd.grad(zp_predict2, zc)
print (zc.grad)
zc.grad.zero_()
grad_zp1_uc = torch.autograd.grad(zp_predict1, uc)
print (uc.grad)
uc.grad.zero_()
grad_zp2_uc = torch.autograd.grad(zp_predict2, uc)
print (uc.grad)
uc.grad.zero_()


tensor([[5., 5., 5., 2., 2.],
        [5., 5., 5., 2., 2.],
        [5., 5., 5., 2., 2.],
        [5., 5., 5., 2., 2.],
        [5., 5., 5., 2., 2.]], grad_fn=<CatBackward>)
tensor([[-0.7835, -5.0028],
        [-0.7835, -5.0028],
        [-0.7835, -5.0028],
        [-0.7835, -5.0028],
        [-0.7835, -5.0028]], grad_fn=<AddmmBackward>)


RuntimeError: grad can be implicitly created only for scalar outputs

In [20]:
N, u_dim, z_dim, out_dim = 5, 2, 3, 2
zc = 5 * torch.ones(N, z_dim, requires_grad=True)
uc = 2 * torch.ones(N, u_dim, requires_grad=True)
input = torch.cat((zc, uc), dim=1)
print (input)
layer = nn.Linear(z_dim + u_dim, out_dim)
output = layer(input)
print (output)
zp_predict1 = output[:, 0]
zp_predict2 = output[:, 1]
grad_zp1_zc = torch.autograd.grad(zp_predict1, zc, grad_outputs=torch.ones([N,]))
print (zc.grad)
zc.grad.zero_()
grad_zp2_zc = torch.autograd.grad(zp_predict2, zc)
print (zc.grad)
zc.grad.zero_()
grad_zp1_uc = torch.autograd.grad(zp_predict1, uc)
print (uc.grad)
uc.grad.zero_()
grad_zp2_uc = torch.autograd.grad(zp_predict2, uc)
print (uc.grad)
uc.grad.zero_()

tensor([[5., 5., 5., 2., 2.],
        [5., 5., 5., 2., 2.],
        [5., 5., 5., 2., 2.],
        [5., 5., 5., 2., 2.],
        [5., 5., 5., 2., 2.]], grad_fn=<CatBackward>)
tensor([[ 2.0257, -0.2668],
        [ 2.0257, -0.2668],
        [ 2.0257, -0.2668],
        [ 2.0257, -0.2668],
        [ 2.0257, -0.2668]], grad_fn=<AddmmBackward>)
None


AttributeError: 'NoneType' object has no attribute 'zero_'

In [75]:
x = torch.randn(7,3)
y = torch.randn(7,3)
stacked = torch.stack([x, y], dim=1)
print (stacked.size())

torch.Size([7, 2, 3])


In [76]:
x = torch.randn(7, 3, 2)
tr = torch.transpose(x, 1, 2)
print (tr.size())

torch.Size([7, 2, 3])


In [87]:
A = torch.randn(7, 3, 2)
inp = torch.randn(7, 2)
print(inp.dim())
res = torch.bmm(A, inp.unsqueeze_(inp.dim()))
print (res.size())

2
torch.Size([7, 3, 1])


In [72]:
# https://discuss.pytorch.org/t/calculate-individual-gradients-for-each-example-in-mini-batch/54470

N, u_dim, z_dim, out_dim = 7, 2, 2, 2
zc = torch.as_tensor(5 * np.ones((N, z_dim)), dtype=torch.float).requires_grad_()
uc = torch.as_tensor(2 * np.ones((N, u_dim)), dtype=torch.float).requires_grad_()
print (zc.is_leaf)
input = torch.cat((zc, uc), dim=1).requires_grad_()
print (input.is_leaf)
print (input)
layer = nn.Linear(z_dim + u_dim, out_dim)
output = layer(input)
print (output)

o0 = output[:, 0]
grad_op0_zc = torch.autograd.grad(o0, zc, grad_outputs=torch.ones(o0.size()), retain_graph=True)
print (grad_op0_zc[0])
o1 = output[:, 1]
grad_op1_zc = torch.autograd.grad(o1, zc, grad_outputs=torch.ones(o1.size()), retain_graph=True)
print (grad_op1_zc[0])

grad_op_zc = torch.autograd.grad(output, zc, grad_outputs=torch.ones(output.size()), retain_graph=False)
print (grad_op_zc[0])

True
False
tensor([[5., 5., 2., 2.],
        [5., 5., 2., 2.],
        [5., 5., 2., 2.],
        [5., 5., 2., 2.],
        [5., 5., 2., 2.],
        [5., 5., 2., 2.],
        [5., 5., 2., 2.]], grad_fn=<CatBackward>)
tensor([[0.0849, 0.9494],
        [0.0849, 0.9494],
        [0.0849, 0.9494],
        [0.0849, 0.9494],
        [0.0849, 0.9494],
        [0.0849, 0.9494],
        [0.0849, 0.9494]], grad_fn=<AddmmBackward>)
tensor([[-0.3552,  0.4388],
        [-0.3552,  0.4388],
        [-0.3552,  0.4388],
        [-0.3552,  0.4388],
        [-0.3552,  0.4388],
        [-0.3552,  0.4388],
        [-0.3552,  0.4388]])
tensor([[ 0.2613, -0.3125],
        [ 0.2613, -0.3125],
        [ 0.2613, -0.3125],
        [ 0.2613, -0.3125],
        [ 0.2613, -0.3125],
        [ 0.2613, -0.3125],
        [ 0.2613, -0.3125]])
tensor([[-0.0939,  0.1263],
        [-0.0939,  0.1263],
        [-0.0939,  0.1263],
        [-0.0939,  0.1263],
        [-0.0939,  0.1263],
        [-0.0939,  0.1263],
        [-0.0